# Task 5.2D: Computer vision and Custom Vision 

GitHub Link: https://github.com/M-S-Kashif/SIT788_Task_5.2D

YouTube Link: 

### Part 1: Install and Import Libraries

In [1]:
!pip install azure-cognitiveservices-vision-customvision

In [2]:
!pip install pillow

In [3]:
#Importing Basic Libraries...
import os
import sys
import glob
import cv2
from PIL import Image, ImageDraw, ImageFont

In [4]:
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateBatch, ImageFileCreateEntry, Region
from msrest.authentication import ApiKeyCredentials
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient

### Part 2: Connection with CustomVisionTrainingClient 

In [5]:
#Initializing all the Credentials...
ENDPOINT = "https://task052d.cognitiveservices.azure.com/"
training_key = "d94e4818839841ca9beb33369f038853"
prediction_key = "acaeaf574b254d8bb7fc4e3258c3b1c1"
prediction_resource_id = "/subscriptions/d5d0e7cd-5900-4c80-820c-b24c2a8416d0/resourceGroups/MUHAMMADSOHAIBBIN-SIT788-2023/providers/Microsoft.CognitiveServices/accounts/Task052D-Prediction"

In [6]:
credentials = ApiKeyCredentials(in_headers={"Training-key": training_key})
trainer = CustomVisionTrainingClient(ENDPOINT, credentials)

### Part 3: Finding the Object Detection Domain

In [7]:
publish_iteration_name = "detectModel"

#Creating an Object Detection Domain...
objdomain = next(domain for domain in trainer.get_domains() if domain.type == "ObjectDetection" and domain.name == "General")
print(objdomain)

{'additional_properties': {}, 'id': 'da2e3a8a-40a5-4171-82f4-58522f70fbc1', 'name': 'General', 'type': 'ObjectDetection', 'exportable': False, 'enabled': True, 'exportable_platforms': [], 'model_information': None}


### Part 4: Create a new Project for Custom Vision

In [8]:
#Create a new Project
print("Creating a new Project...")
project = trainer.create_project("VehicleDetectorCV_v2", domain_id=objdomain.id)
print("\n\n\n Project Created: ", project.id)

Creating a new Project...



 Project Created:  ce913947-436a-4275-9a9e-aab6919c8d83


In [9]:
# project = trainer.get_project

### Part 5: Creating tags in a project

In [10]:
#Making tags for different types of vehicles...
h_tag = trainer.create_tag(project.id, "hatchback")
s_tag = trainer.create_tag(project.id, "sedan")
F_tag = trainer.create_tag(project.id, "SUV")
b_tag = trainer.create_tag(project.id, "bus")

### Part 6: Uploading the images and their tags into the Trainer

"As the above title suggests, we have uploaded both the images and their tags for our trainer. So we have sub-divided this project into two domains. They are mentioned as below: "

#### i- Uploading the Image Tags

In [11]:
b_image_regions = {    #filename : regions
    "b1": [ 0.145, 0.230, 0.684, 0.705],
    "b2": [ 0.120, 0.120, 0.856, 0.856],
    "b3": [ 0.145, 0.230, 0.784, 0.705],
    "b4": [ 0.155, 0.180, 0.824, 0.765],
    "b5": [ 0.161, 0.141, 0.834, 0.865],
    "b6": [ 0.0, 0.161, 0.984, 0.805],
    "b7": [ 0.01, 0.130, 0.984, 0.865],
    "b8": [ 0.01, 0.08, 0.985, 0.905],
    "b9": [ 0.05, 0.03, 0.894, 0.965],
    "b10": [ 0.06, 0.04, 0.984, 0.925],
    "b11": [ 0.25, 0.08, 0.654, 0.865],
    "b12": [ 0.05, 0.02, 0.964, 0.965],
    "b13": [ 0.03, 0.01, 0.944, 0.970],
    "b14": [ 0.07, 0.05, 0.894, 0.865],
    "b15": [ 0.15, 0.13, 0.834, 0.785]
}

In [12]:
h_image_regions = {
    "h1": [ 0.160, 0.337, 0.694, 0.505],
    "h2": [ 0.160, 0.0, 0.836, 0.894],
    "h3": [ 0.135, 0.210, 0.684, 0.715],
    "h4": [ 0.2, 0.0, 0.724, 0.965],
    "h5": [ 0.161, 0.141, 0.834, 0.865],
    "h6": [ 0.0, 0.161, 0.984, 0.805],
    "h7": [ 0.11, 0.330, 0.884, 0.665],
    "h8": [ 0.01, 0.08, 0.985, 0.905],
    "h9": [ 0.05, 0.07, 0.894, 0.865],
    "h10": [ 0.06, 0.06, 0.884, 0.825],
    "h11": [ 0.05, 0.03, 0.894, 0.965],
    "h12": [ 0.12, 0.21, 0.794, 0.703],
    "h13": [ 0.11, 0.12, 0.794, 0.768],
    "h14": [ 0.15, 0.33, 0.833, 0.84],
    "h15": [ 0.14, 0.23, 0.834, 0.685]
}

In [13]:
s_image_regions = {
    "s1": [ 0.101, 0.087, 0.834, 0.705],
    "s2": [ 0.022, 0.326, 0.956, 0.556],
    "s3": [ 0.025, 0.214, 0.954, 0.805],
    "s4": [ 0.252, 0.380, 0.654, 0.665],
    "s5": [ 0.161, 0.141, 0.834, 0.865],
    "s6": [ 0.0, 0.361, 0.984, 0.605],
    "s7": [ 0.05, 0.430, 0.944, 0.565],
    "s8": [ 0.08, 0.28, 0.915, 0.405],
    "s9": [ 0.09, 0.11, 0.824, 0.745],
    "s10": [ 0.06, 0.04, 0.984, 0.925],
    "s11": [ 0.11, 0.194, 0.794, 0.765],
    "s12": [ 0.08, 0.08, 0.824, 0.766],
    "s13": [ 0.32, 0.312, 0.594, 0.465],
    "s14": [ 0.05, 0.03, 0.894, 0.965],
    "s15": [ 0.15, 0.13, 0.834, 0.785]
}

In [14]:
F_image_regions = {
    "F1": [ 0.21, 0.030, 0.684, 0.935],
    "F2": [ 0.120, 0.320, 0.756, 0.444],
    "F3": [ 0.145, 0.230, 0.784, 0.705],
    "F4": [ 0.245, 0.280, 0.724, 0.715],
    "F5": [ 0.160, 0.291, 0.834, 0.673],
    "F6": [ 0.0, 0.231, 0.999, 0.753],
    "F7": [ 0.111, 0.313, 0.584, 0.565],
    "F8": [ 0.207, 0.189, 0.784, 0.705],
    "F9": [ 0.09, 0.223, 0.814, 0.645],
    "F10": [ 0.04, 0.04, 0.984, 0.985],
    "F11": [ 0.25, 0.191, 0.694, 0.785],
    "F12": [ 0.25, 0.47, 0.394, 0.365],
    "F13": [ 0.08, 0.112, 0.874, 0.698],
    "F14": [ 0.05, 0.37, 0.894, 0.335],
    "F15": [ 0.148, 0.189, 0.834, 0.685]
}

In [15]:
base_image_url = "C:/Users/Sohaib bin Kashif/OneDrive/Desktop/SIT788 - Engineering AI Solutions/OnTrack Tasks/Week 5/Task 5.2D/"
print("Path: ", base_image_url)

Path:  C:/Users/Sohaib bin Kashif/OneDrive/Desktop/SIT788 - Engineering AI Solutions/OnTrack Tasks/Week 5/Task 5.2D/


#### ii- Uploading the Images

"In this step, we also added the images in the Custom Vision Website's UI, where we can then use to train the model. We have managed to add 15 images of four types of vehicles (making it 60 images in total). The four types of vehicles we will assess are SUVs, Buses, Hatchbacks, and Sedans."

In [16]:
#Adding all the tags from our list...
tagged_images_with_h_regions = []

for file_name in h_image_regions.keys():
    print(file_name)
    x,y,w,h = h_image_regions[file_name]
    regions = [ Region(tag_id = h_tag.id, left=x, top=y, width=w, height=h)]
    
    with open(base_image_url + "vehicle_images/hatchback/" + file_name + ".png", mode="rb") as image_data:
        tagged_images_with_h_regions.append(ImageFileCreateEntry(name=file_name,\
                                                               contents=image_data,\
                                                               regions=regions))
        
#Adding our Batch into the Custom Vision UI...
image_paths = glob.glob(base_image_url + "vehicle_images/hatchback/" + "/*.png")
image_list = []

for image_path in image_paths:
    with open(image_path, "rb") as image_contents:
        image_list.append(ImageFileCreateEntry(name=image_path, contents=image_contents.read()))

batch = ImageFileCreateBatch(images=image_list)
batch.tag_ids = [h_tag.id]

upload_result = trainer.create_images_from_files(project.id, batch)
print("All of the Hatchbacks added...")

h1
h2
h3
h4
h5
h6
h7
h8
h9
h10
h11
h12
h13
h14
h15
All of the Hatchbacks added...


In [17]:
#Adding all the tags from our list...
tagged_images_with_s_regions = []

for file_name in s_image_regions.keys():
    print(file_name)
    x,y,w,h = s_image_regions[file_name]
    regions = [ Region(tag_id = s_tag.id, left=x, top=y, width=w, height=h)]
    
    with open(base_image_url + "vehicle_images/sedan/" + file_name + ".png", mode="rb") as image_contents:
        tagged_images_with_s_regions.append(ImageFileCreateEntry(name=file_name, contents=image_contents.read(),\
                                         regions=regions))
        
#Adding our Batch into the Custom Vision UI...        
image_paths = glob.glob(base_image_url + "vehicle_images/sedan/" + "/*.png")
image_list = []

for image_path in image_paths:
    with open(image_path, "rb") as image_contents:
        image_list.append(ImageFileCreateEntry(name=image_path, contents=image_contents.read()))

batch = ImageFileCreateBatch(images=image_list)
batch.tag_ids = [s_tag.id]

upload_result = trainer.create_images_from_files(project.id, batch)
print("All of the Sedans added...")

s1
s2
s3
s4
s5
s6
s7
s8
s9
s10
s11
s12
s13
s14
s15
All of the Sedans added...


In [18]:
#Adding all the tags from our list...
tagged_images_with_F_regions = []

for file_name in F_image_regions.keys():
    print(file_name)
    x,y,w,h = F_image_regions[file_name]
    regions = [ Region(tag_id = F_tag.id, left=x, top=y, width=w, height=h)]
    
    with open(base_image_url + "vehicle_images/SUV/" + file_name + ".png", mode="rb") as image_contents:
        tagged_images_with_F_regions.append(ImageFileCreateEntry(name=file_name, contents=image_contents.read(),\
                                         regions=regions))

#Adding our Batch into the Custom Vision UI...        
image_paths = glob.glob(base_image_url + "vehicle_images/SUV/" + "/*.png")
image_list = []

for image_path in image_paths:
    with open(image_path, "rb") as image_contents:
        image_list.append(ImageFileCreateEntry(name=image_path, contents=image_contents.read()))

batch = ImageFileCreateBatch(images=image_list)
batch.tag_ids = [F_tag.id]

upload_result = trainer.create_images_from_files(project.id, batch)
print("All of the SUVs added...")

F1
F2
F3
F4
F5
F6
F7
F8
F9
F10
F11
F12
F13
F14
F15
All of the SUVs added...


In [19]:
#Adding all the tags from our list...
tagged_images_with_b_regions = []

for file_name in b_image_regions.keys():
    print(file_name)
    x,y,w,h = b_image_regions[file_name]
    regions = [ Region(tag_id = b_tag.id, left=x, top=y, width=w, height=h)]
    
    with open(base_image_url + "vehicle_images/bus/" + file_name + ".png", mode="rb") as image_contents:
        tagged_images_with_b_regions.append(ImageFileCreateEntry(name=file_name, contents=image_contents.read(),\
                                         regions=regions))
        
#Adding our Batch into the Custom Vision UI...        
image_paths = glob.glob(base_image_url + "vehicle_images/bus/" + "/*.png")
image_list = []

for image_path in image_paths:
    with open(image_path, "rb") as image_contents:
        image_list.append(ImageFileCreateEntry(name=image_path, contents=image_contents.read()))

batch = ImageFileCreateBatch(images=image_list)
batch.tag_ids = [b_tag.id]

upload_result = trainer.create_images_from_files(project.id, batch)
print("All of the buses added...")

b1
b2
b3
b4
b5
b6
b7
b8
b9
b10
b11
b12
b13
b14
b15
All of the buses added...


**Note:** From this point, please ensure that you have tagged all the images added into the model before training and testing.

### Part 7: Training the Classifier

In [20]:
print("Training our Model...")
iteration = trainer.train_project(project.id)

while (iteration.status != "Completed"):
    iteration = trainer.get_iteration(project.id, iteration.id)
    print("Training Status: ", iteration.status)
    #time.sleep(1)

Training our Model...
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Traini

Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
T

Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
T

Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
T

Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
T

Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
T

Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
T

Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
Training Status:  Training
T

In [21]:
trainer.publish_iteration(project.id, iteration.id,\
                          publish_iteration_name, prediction_resource_id)
print("All Done...")

All Done...


### Part 8: Testing and Publishing the Classifier

In [22]:
#Connecting our Prediction Client...
ENDPOINT= "https://task052d-prediction.cognitiveservices.azure.com"
prediction_key = "acaeaf574b254d8bb7fc4e3258c3b1c1"

prediction_credentials = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key})
predictor = CustomVisionPredictionClient(ENDPOINT, prediction_credentials)

In [23]:
with open(base_image_url + "vehicle_images/test.png", mode='rb') as test_data:
    results = predictor.detect_image(project.id, publish_iteration_name, test_data)

#### i- Finding the Predictions of the Model.

In [24]:
#Display the results.
for prediction in results.predictions:
    print("\t" + prediction.tag_name + ": {0:.2f}% bbox.left = {1:.2f}, bbox.top = {2:.2f}, bbox.width = {3:.2f}, bbox.height = {4:.2f}".format(prediction.probability * 100, prediction.bounding_box.left, prediction.bounding_box.top, prediction.bounding_box.width, prediction.bounding_box.height))

	hatchback: 2.13% bbox.left = 0.36, bbox.top = 0.00, bbox.width = 0.64, bbox.height = 0.94
	hatchback: 1.96% bbox.left = 0.13, bbox.top = 0.52, bbox.width = 0.15, bbox.height = 0.37
	sedan: 1.75% bbox.left = 0.13, bbox.top = 0.52, bbox.width = 0.15, bbox.height = 0.37
	sedan: 1.58% bbox.left = 0.38, bbox.top = 0.56, bbox.width = 0.46, bbox.height = 0.44
	bus: 1.29% bbox.left = 0.36, bbox.top = 0.00, bbox.width = 0.64, bbox.height = 0.94
	sedan: 1.10% bbox.left = 0.36, bbox.top = 0.05, bbox.width = 0.64, bbox.height = 0.95
	sedan: 0.99% bbox.left = 0.09, bbox.top = 0.64, bbox.width = 0.06, bbox.height = 0.15
	hatchback: 0.95% bbox.left = 0.09, bbox.top = 0.64, bbox.width = 0.06, bbox.height = 0.15
	hatchback: 0.92% bbox.left = 0.13, bbox.top = 0.74, bbox.width = 0.05, bbox.height = 0.10
	hatchback: 0.90% bbox.left = 0.63, bbox.top = 0.52, bbox.width = 0.16, bbox.height = 0.39
	sedan: 0.87% bbox.left = 0.63, bbox.top = 0.52, bbox.width = 0.16, bbox.height = 0.39
	hatchback: 0.86% bbox.le

#### ii- Finding the Top 5 regions from the Model. 

"Given all of these overlapping predictions, we can see that there are numerous regions have been made in regards to the test image. But as we all know, the model will consider only the region with the most probability. In our case, the model detects a car for a sedan in 'test.png'. Let's take the top five predictions from the model aand draw some rectangles over the image."

In [25]:
predicted_regions = []
region_names = []
i = 0

for prediction in results.predictions:
    print("\t" + prediction.tag_name + ": {0:.2f}% bbox.left = {1:.2f}, bbox.top = {2:.2f}, bbox.width = {3:.2f}, bbox.height = {4:.2f}".format(prediction.probability * 100, prediction.bounding_box.left, prediction.bounding_box.top, prediction.bounding_box.width, prediction.bounding_box.height))
    l = prediction.bounding_box.left
    t = prediction.bounding_box.top
    w = prediction.bounding_box.width
    h = prediction.bounding_box.height

    region = prediction.tag_name
    region_summary = [l, t, w, h]
    region_names.append(region)
    predicted_regions.append(region_summary)
    i = i + 1
    
    if (i == 5):
        break
      
print("\n\n Regions: \n", region_names)    
print("\n\n Predicted Regions: \n",predicted_regions)

	hatchback: 2.13% bbox.left = 0.36, bbox.top = 0.00, bbox.width = 0.64, bbox.height = 0.94
	hatchback: 1.96% bbox.left = 0.13, bbox.top = 0.52, bbox.width = 0.15, bbox.height = 0.37
	sedan: 1.75% bbox.left = 0.13, bbox.top = 0.52, bbox.width = 0.15, bbox.height = 0.37
	sedan: 1.58% bbox.left = 0.38, bbox.top = 0.56, bbox.width = 0.46, bbox.height = 0.44
	bus: 1.29% bbox.left = 0.36, bbox.top = 0.00, bbox.width = 0.64, bbox.height = 0.94


 Regions: 
 ['hatchback', 'hatchback', 'sedan', 'sedan', 'bus']


 Predicted Regions: 
 [[0.36370948, 0.0, 0.6362895, 0.943598], [0.13285477, 0.5214912, 0.1521195, 0.36749095], [0.13285477, 0.5214912, 0.1521195, 0.36749095], [0.37709838, 0.5602217, 0.46307647, 0.4397773], [0.36370948, 0.0, 0.6362895, 0.943598]]


#### iiI- Appending the object boundaries

"Now, let's take a small screenshot from a video clip and use this as a test image for our model..."

In [26]:
def draw_object_boundaries(im, region_name, x1, y1, x, y):
    #im = Image.open(img)
    draw = ImageDraw.Draw(im)
    draw.rectangle((x1, y1, x, y), outline="red",width=3)
    draw.text((x + 5, y + 5), region_name, fill =(255, 0, 0), size=40)
    print("Object Drawn... \n")

In [27]:
image_name = 'vehicle_images/test.png'
print("\n\nImage Name: ", image_name)
im = Image.open(image_name)  
print("Image Opened")

for i in range(0,5):
    #im = Image.open(img)
    l, t, w, h = predicted_regions[i]
    print("Normalized Coordinates: ",(l, t, w, h))
    
    x = l * im.width
    x1 = (l + w) * im.width
    y = t * im.height
    y1 = (t + h) * im.height
    print("Image Coordinates: ",(x, y, w, h))
            
    #Drawing Boundaries for the Objects...
    draw_object_boundaries(im, region_names[i], x1, y1, x, y)
      
    im.save("vehicle_images/test_result.png")

im.close()    
print("New Image with Detected regions Saved...")



Image Name:  vehicle_images/test.png
Image Opened
Normalized Coordinates:  (0.36370948, 0.0, 0.6362895, 0.943598)
Image Coordinates:  (391.71510995999995, 0.0, 0.6362895, 0.943598)
Object Drawn... 

Normalized Coordinates:  (0.13285477, 0.5214912, 0.1521195, 0.36749095)
Image Coordinates:  (143.08458729, 298.8144576, 0.1521195, 0.36749095)
Object Drawn... 

Normalized Coordinates:  (0.13285477, 0.5214912, 0.1521195, 0.36749095)
Image Coordinates:  (143.08458729, 298.8144576, 0.1521195, 0.36749095)
Object Drawn... 

Normalized Coordinates:  (0.37709838, 0.5602217, 0.46307647, 0.4397773)
Image Coordinates:  (406.13495526, 321.0070341, 0.46307647, 0.4397773)
Object Drawn... 

Normalized Coordinates:  (0.36370948, 0.0, 0.6362895, 0.943598)
Image Coordinates:  (391.71510995999995, 0.0, 0.6362895, 0.943598)
Object Drawn... 

New Image with Detected regions Saved...


"Now that we know that our model can detect images effectively, let's create our vehicle tracker!"

### Part 9: Near Real-time Vehicle Tracker

#### i- Capturing a Frame from the Video Clip.

In [56]:
#Creating a folder for our Images...
path = base_image_url + 'data'
if os.path.exists(path):
    os.makedirs('data/')

In [58]:
#Checking the path again...
os.path.exists(path)

True

In [62]:
#Creating a VideoCapture object from OpenCV Libraries...
video_path = base_image_url + 'vehicle_images/car.mp4'
vidcap = cv2.VideoCapture(video_path)
out_dir = base_image_url + 'data'
numframe = 1

while True:
# while vidcap.isOpened():
#     time.sleep(delay)  #Assume we capture something after 3 seconds...
    _ , frame = vidcap.read()   #Capturing the frame...
    cv2.imshow('capture',frame)
    
    #Capturing the frame from the video...
    if cv2.waitKey(32) & 0xFF == ord('a'):
        print("Capture!")
        cap_frame = cv2.imwrite(out_dir + "/frame" + str(numframe) +'.png', frame)
        cv2.imshow('Captured Frame', cap_frame)
        numframe += 1
      
#     if success:
#         print("Capture!")
#         frame = cv2.imwrite(out_dir + str(currentframe) +'.png', frame)
#         currentframe += 1
#         cv2.imshow('Capture', frame)
        
#         if cv2.waitKey(32) & 0xFF == ord('q'):
#             vidcap.release()
#             break

    if cv2.waitKey(32) & 0xFF == ord('q'):
        print("End of Video Capture...")
        print ("{} frame(s) Captured".format(numframe - 1))
        break

vidcap.release()        
cv2.destroyAllWindows()

Capture!
Capture!
Capture!
Capture!
Capture!
Capture!
Capture!
Capture!
Capture!
Capture!
Capture!
Capture!
End of Video Capture...
13 frame(s) Captured


"The above code is capable of taking numerous instances and from it we can use the best possible image to identify the car itself. Our next block of code will only use a single frame to determine the type of a car."

#### ii- Running our Vehicle Tracker feature onto the capture frame

In [71]:
#Creating our function to test and then print results...

def vehicle_tracker(video_frame_img):
    
    #Step 1: Opening Data Frame in the model to get Detection results...
    with open(base_image_url + video_frame_img , mode='rb') as video_frame:
        results = predictor.detect_image(project.id, publish_iteration_name, video_frame)
        
    #Step 2: Capturing the Top 5 predicted regions...
    predicted_regions = []
    region_names = []
    i = 0

    for prediction in results.predictions:
        print("\t" + prediction.tag_name + ": {0:.2f}% bbox.left = {1:.2f}, bbox.top = {2:.2f}, bbox.width = {3:.2f}, bbox.height = {4:.2f}".format(prediction.probability * 100, prediction.bounding_box.left, prediction.bounding_box.top, prediction.bounding_box.width, prediction.bounding_box.height))
        l = prediction.bounding_box.left
        t = prediction.bounding_box.top
        w = prediction.bounding_box.width
        h = prediction.bounding_box.height

        region = prediction.tag_name
        region_summary = [l, t, w, h]
        region_names.append(region)
        predicted_regions.append(region_summary)
        i = i + 1

        if (i == 5):
            break

    print("\n\n Regions: \n", region_names)    
    print("\n\n Predicted Regions: \n",predicted_regions)
    
    #Step 3: Use the pillow library to create a new image with detected regions...
    im = Image.open(video_frame_img)  
    print("Image Opened")

    for i in range(0,5):
        #im = Image.open(img)
        l, t, w, h = predicted_regions[i]
        print("Normalized Coordinates: ",(l, t, w, h))

        x = l * im.width
        x1 = (l + w) * im.width
        y = t * im.height
        y1 = (t + h) * im.height
        print("Image Coordinates: ",(x, y, w, h))

        #Drawing Boundaries for the Objects...
        draw_object_boundaries(im, region_names[i], x1, y1, x, y)

        im.save("vehicle_images/capture_frame.png")

    im.close()    
    print("New Image with Detected regions Saved...")

In [65]:
# Finally running our vehicle tracker function...
video_frame = "vehicle_images/test.png"
vehicle_tracker(video_frame)

	hatchback: 2.13% bbox.left = 0.36, bbox.top = 0.00, bbox.width = 0.64, bbox.height = 0.94
	hatchback: 1.96% bbox.left = 0.13, bbox.top = 0.52, bbox.width = 0.15, bbox.height = 0.37
	sedan: 1.75% bbox.left = 0.13, bbox.top = 0.52, bbox.width = 0.15, bbox.height = 0.37
	sedan: 1.58% bbox.left = 0.38, bbox.top = 0.56, bbox.width = 0.46, bbox.height = 0.44
	bus: 1.29% bbox.left = 0.36, bbox.top = 0.00, bbox.width = 0.64, bbox.height = 0.94


 Regions: 
 ['hatchback', 'hatchback', 'sedan', 'sedan', 'bus']


 Predicted Regions: 
 [[0.36370948, 0.0, 0.6362895, 0.943598], [0.13285477, 0.5214912, 0.1521195, 0.36749095], [0.13285477, 0.5214912, 0.1521195, 0.36749095], [0.37709838, 0.5602217, 0.46307647, 0.4397773], [0.36370948, 0.0, 0.6362895, 0.943598]]
Image Opened
Normalized Coordinates:  (0.36370948, 0.0, 0.6362895, 0.943598)
Image Coordinates:  (391.71510995999995, 0.0, 0.6362895, 0.943598)
Object Drawn... 

Normalized Coordinates:  (0.13285477, 0.5214912, 0.1521195, 0.36749095)
Image Coor

In [72]:
# Finally running our vehicle tracker function...
video_frame = "data/frame6.png"
vehicle_tracker(video_frame)

	sedan: 10.96% bbox.left = 0.10, bbox.top = 0.57, bbox.width = 0.47, bbox.height = 0.42
	hatchback: 6.21% bbox.left = 0.10, bbox.top = 0.57, bbox.width = 0.47, bbox.height = 0.42
	hatchback: 3.66% bbox.left = 0.37, bbox.top = 0.00, bbox.width = 0.63, bbox.height = 0.93
	bus: 2.91% bbox.left = 0.37, bbox.top = 0.00, bbox.width = 0.63, bbox.height = 0.93
	sedan: 1.30% bbox.left = 0.37, bbox.top = 0.00, bbox.width = 0.63, bbox.height = 0.93


 Regions: 
 ['sedan', 'hatchback', 'hatchback', 'bus', 'sedan']


 Predicted Regions: 
 [[0.09996018, 0.57378685, 0.47424307, 0.4229983], [0.09996018, 0.57378685, 0.47424307, 0.4229983], [0.37317702, 0.0, 0.626822, 0.92542243], [0.37317702, 0.0, 0.626822, 0.92542243], [0.37317702, 0.0, 0.626822, 0.92542243]]
Image Opened
Normalized Coordinates:  (0.09996018, 0.57378685, 0.47424307, 0.4229983)
Image Coordinates:  (127.9490304, 413.126532, 0.47424307, 0.4229983)
Object Drawn... 

Normalized Coordinates:  (0.09996018, 0.57378685, 0.47424307, 0.4229983)


### References:

- https://www.youtube.com/watch?v=gigbdHOigtE
- https://learn.microsoft.com/en-us/azure/cognitive-services/custom-vision-service/quickstarts/object-detection?tabs=visual-studio&pivots=programming-language-python
- https://towardsdatascience.com/create-a-model-with-azure-custom-vision-and-python-7bc5caed82c4
- https://learn.microsoft.com/en-us/azure/cognitive-services/Custom-Vision-Service/quickstarts/image-classification?tabs=visual-studio&pivots=programming-language-python
- https://stackoverflow.com/questions/30136257/how-to-get-image-from-video-using-opencv-python
- https://www.youtube.com/watch?v=Ay6Nlb3KJog&t=32s
- https://github.com/Ai-Genome-Saksham/OpenCV/blob/main/OpenCV/%231.2%20Video%20Capture%20Object%20creation.py
